In [1]:
! pip install transformers peft torch accelerate

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType, PeftModel

# Load the tokenizer and model (LLaMA 2-3B from Hugging Face)
model_name = "meta-llama/Llama-3.2-1B-Instruct"
hf_token = ""
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token)

# Add a padding token to the tokenizer if it doesn't have one
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# Configure LoRA parameters
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # For causal language modeling
    r=16,                          # Rank of the LoRA updates
    lora_alpha=32,                 # Scaling factor for LoRA
    lora_dropout=0.1,              # Dropout for LoRA layers
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Set up training configurations
training_args = TrainingArguments(
    output_dir="./lora-llama-2-3b-unsupervised",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    save_steps=500,
    evaluation_strategy="no",
    report_to=[],
    save_safetensors=False
)

# Prepare a simple self-supervised dataset for unsupervised fine-tuning
# Assuming you have raw text data in a file named 'data.txt' for simplicity
import pandas as pd
from tqdm import tqdm
dataset = pd.read_csv('interviewer.csv', encoding='Windows-1252')

# Tokenize the dataset for LLaMA (causal language modeling task)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tqdm.pandas(desc="Processing Data")
tokenized_datasets = dataset.progress_apply(lambda x: tokenize_function(x), axis=1)

# Define data collator for causal LM (shifts labels for next token prediction)
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialize Trainer for LoRA fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Processing Data: 100%|██████████| 5/5 [00:00<00:00, 348.25it/s]


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-671d65ae-39f2305f40b095013fb8c95c;48fdc100-fc68-4745-a231-4fb702288c46)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B-Instruct is restricted. You must have access to it and be authenticated to access 

TrainOutput(global_step=3, training_loss=1.4189856847127278, metrics={'train_runtime': 14.2219, 'train_samples_per_second': 1.055, 'train_steps_per_second': 0.211, 'total_flos': 44921090211840.0, 'train_loss': 1.4189856847127278, 'epoch': 3.0})

In [3]:
model.save_pretrained("./lora-llama-2-3b-unsupervised")
tokenizer.save_pretrained("./lora-llama-2-3b-unsupervised")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-671d65af-5bb5035c09c0a5b81e2e8448;60f06a3e-e0d8-47e0-ab7a-3221dacf36e9)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B-Instruct.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in meta-llama/Llama-3.2-1B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


('./lora-llama-2-3b-unsupervised/tokenizer_config.json',
 './lora-llama-2-3b-unsupervised/special_tokens_map.json',
 './lora-llama-2-3b-unsupervised/tokenizer.json')

In [4]:
! zip -r lora-llama-2-3b-unsupervised.zip lora-llama-2-3b-unsupervised

  adding: lora-llama-2-3b-unsupervised/ (stored 0%)
  adding: lora-llama-2-3b-unsupervised/adapter_model.safetensors (deflated 10%)
  adding: lora-llama-2-3b-unsupervised/README.md (deflated 66%)
  adding: lora-llama-2-3b-unsupervised/tokenizer_config.json (deflated 94%)
  adding: lora-llama-2-3b-unsupervised/checkpoint-3/ (stored 0%)
  adding: lora-llama-2-3b-unsupervised/checkpoint-3/README.md (deflated 66%)
  adding: lora-llama-2-3b-unsupervised/checkpoint-3/optimizer.pt (deflated 7%)
  adding: lora-llama-2-3b-unsupervised/checkpoint-3/training_args.bin (deflated 51%)
  adding: lora-llama-2-3b-unsupervised/checkpoint-3/scheduler.pt (deflated 56%)
  adding: lora-llama-2-3b-unsupervised/checkpoint-3/trainer_state.json (deflated 55%)
  adding: lora-llama-2-3b-unsupervised/checkpoint-3/rng_state.pth (deflated 25%)
  adding: lora-llama-2-3b-unsupervised/checkpoint-3/adapter_config.json (deflated 50%)
  adding: lora-llama-2-3b-unsupervised/checkpoint-3/adapter_model.bin (deflated 10%)
  a

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the fine-tuned model and tokenizer
model_name = "./lora-llama-2-3b-unsupervised"  # Path to the fine-tuned model directory
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map='auto')
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to simulate the interview
def interview_simulation(model, tokenizer):
    # Initial context to set the tone for the interview
    context = "Interviewee: Hello, I'm ready for the interview.\nInterviewer: Welcome! Let's start with some questions about Django and Flask.\n"

    # First question to start the interview
    interviewer_question = "To start, can you describe how Django manages user sessions?"
    print(f"Interviewer: {interviewer_question}")
    context += f"Interviewer: {interviewer_question}\n"

    # Start the interactive loop
    while True:
        torch.cuda.empty_cache()
        # Interviewee's input
        Interviewee_response = input("Interviewee: ")
        context += f"Interviewee: {Interviewee_response}\n"

        # Prepare context for next interviewer question
        context += "Interviewer: "

        # Tokenize and generate the next question based on the context
        inputs = tokenizer(context, return_tensors="pt")
        attention_mask = inputs["attention_mask"].to(device)
        input_ids = inputs["input_ids"].to(device)
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                # max_length=512,
                pad_token_id=tokenizer.eos_token_id,
                attention_mask=attention_mask,
                num_return_sequences=1,
                no_repeat_ngram_size=3,
                max_new_tokens=8,
                temperature=0.7  # Control creativity for follow-ups
            )

        # Decode the generated question
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        next_interviewer_question = generated_text.split("Interviewer:")[-1].strip().split("Interviewee:")[0].strip()

        # Print and update context
        print(f"Interviewer: {next_interviewer_question}")
        context += f"{next_interviewer_question}\n"

        # Exit condition for the loop
        if Interviewee_response.lower() in ["exit", "quit"]:
            print("Interviewer: Thank you for your time. This concludes the interview.")
            break

# Run the interview simulation
interview_simulation(model, tokenizer)

Interviewer: To start, can you describe how Django manages user sessions?
Interviewee: yes
Interviewer: Can you explain how Django handles session data
Interviewee: no i can't
Interviewer: Can you describe the difference between a session
Interviewee: none
Interviewer: Can you tell me about the role of
Interviewee: nothing
Interviewer: How do you handle session expiration?
Interview
Interviewee: i can't
Interviewer: Can you provide an example of how to
Interviewee: no baby
Interviewer: In this case, I would like to
Interviewee: what
Interviewer: 2.3.4.5
Interviewee: trtr
Interviewer: 3.2.5.6
Interviewee: ask more question
Interviewer: 4.4
Interviewe
Interview
Interviewee: exit
Interviewer: 6
This interview was a disaster.
Interviewer: Thank you for your time. This concludes the interview.
